
# Calculate NRCS-CN value

This notebook calculates the NRCS CN for each watershed by clipping data from this dataset:

https://figshare.com/articles/dataset/GCN250_global_curve_number_datasets_for_hydrologic_modeling_and_design/7756202

which is documented in the following paper https://www.nature.com/articles/s41597-019-0155-x

In [0]:
#S3 bucket information or DBFS
mount_name  = "lwi-transition-zone" #lwi-transition-zone OR jacksonville-data
aws_bucket_name = "lwi-transition-zone" #lwi-transition-zone OR jacksonville-data

local_dir_path = "/local_disk0"
directory_path_data="data/hydrology/USGS_gage_parameters"

directory_path_data_CN= 'data/hydrology/CN'
file_name_CN='GCN250_ARCII.tif'
raster_path=f"/dbfs/mnt/jacksonville-data/{directory_path_data_CN}/{file_name_CN}"

directory_for_baseflow_data=f"data/hydrology/baseflow"
local_dir_path_baseflow = f'{local_dir_path}/hydrology/baseflow'
s3_bucket_baseflow_dir = f"/dbfs/mnt/{aws_bucket_name}/{directory_for_baseflow_data}"

valid_sites_indices = [5,9,12,17,18,19,21,23,25,27,28,29,35,36,38,39,40,43,45,46,47,58,59,63,68]


#Jacksonville valid sites [0,1,2,3,4,5,7,8,9,11,12,13,14,16,17,18,19,20,21,22,23,24,25,26,28,31,32,33,37,38,39,40, 53, 54, 55, 56, 59, 60, 62, 63, 64, 65, 68,69, 70,71, 73,74, 75, 76, 77, 78, 79, 81, 83, 84]
#LWI valid sites [5,9,12,17,18,19,21,23,25,27,28,29,35,36,38,39,40,43,45,46,47,58,59,63,68]

#### Import Libraries

In [0]:
%load_ext autoreload
%autoreload 2

from src.data.utils_s3 import *
from src.data.utils import *
from src.data.utils_files import *
import os, shutil, pickle
import rioxarray as rxr
from src.data.utils_geo import *


#### Mount s3 Bucket
Mounting the s3 bucket is important because it extends the local files system. Without the bucket, the downloaded files may exceed the storage capacity of the computer.

Here we mount the bucket with an instance profile, so the selected cluster must have the appropriate instance profile selected, which in this case is 'ec2-s3-access-role'. For reference see: 
- https://docs.databricks.com/en/aws/iam/instance-profile-tutorial.html
- https://docs.databricks.com/en/dbfs/mounts.html

Here, we also check for the directories where we will store files and create any missing directories.

In [0]:
mount_aws_bucket(aws_bucket_name, mount_name)

%md 
#### Move Data to the Local Drive

In [0]:
create_dir_list = [local_dir_path_baseflow]
create_directories(create_dir_list)
for file_name in os.listdir(s3_bucket_baseflow_dir):
    if os.path.exists(f"/local_disk0/{file_name}")!=1:
        file_path = os.path.join(s3_bucket_baseflow_dir, file_name)
        if os.path.isfile(file_path):
            shutil.copy(file_path, os.path.join(local_dir_path_baseflow, file_name))

%md 
### Load Gage Data

In [0]:
directory = local_dir_path_baseflow
with open(directory+'/usgs_gage_data.pickle', 'rb') as file:
    clipped_gages = pickle.load(file)

### Get Basin Boundaries

In [0]:
clipped_gages_huc12_huc10 = clipped_gages[(pd.to_numeric(clipped_gages['drain_area_va'], errors='coerce').astype(float)< 298.0) & (pd.to_numeric(clipped_gages['drain_area_va'], errors='coerce').astype(float) > 2.0)]
clipped_gages_huc12_huc10_valid = clipped_gages_huc12_huc10.loc[valid_sites_indices]
clipped_gages_huc12_huc10_valid['geometry2']=None
clipped_gages_huc12_huc10_valid.set_geometry('geometry2', inplace=False).set_crs('EPSG:4326')
clipped_gages_huc12_huc10_valid['geometry2'] = clipped_gages_huc12_huc10_valid['site_no'].apply(get_basin_geometry)

### Clip CN Dataset to Study Area

In [0]:


# Load raster
raster = rxr.open_rasterio(raster_path,masked=True,chunks=True)# masked=True,  chunks=True

gdf = clipped_gages_huc12_huc10_valid.set_geometry("geometry2").set_crs("EPSG:4326")

raster_crs = raster.rio.crs

gdf_reprojected = gdf.to_crs(raster_crs)

bbox = gdf_reprojected.total_bounds  # (minx, miny, maxx, maxy)
raster_clipped = raster.rio.clip_box(minx=bbox[0], miny=bbox[1], maxx=bbox[2], maxy=bbox[3])

#### Plot the Watershed and CN data

In [0]:
# Plot the raster
fig, ax = plt.subplots(figsize=(10, 8))
raster_clipped.plot(ax=ax, cmap="terrain")  # Adjust cmap as needed

# Plot the GeoDataFrame on top
gdf_reprojected.plot(ax=ax, facecolor='none', edgecolor='red', linewidth=1.5)

# Show the plot
plt.show()

#gdf_reprojected.iloc[0:1].plot()

### Calculate Average CN for each Watershed

In [0]:
average_values = []
for huc8_in in gdf_reprojected.huc_cd.unique(): #[4:5]
    #print(huc8_in)
    for _, row in gdf_reprojected[gdf_reprojected.huc_cd==huc8_in].iterrows():
        clipped = raster_clipped.rio.clip([row.geometry2], gdf.crs)
        mean_value =np.nanmean(clipped.values)  # Convert to scalar
        average_values.append(mean_value)
        print(mean_value)